# ATR breakount - Analytics

### Import Library

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [3]:
data = dataset.copy()

In [4]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765
...,...,...,...,...,...
2025-02-07 14:15:00,1343.3,1344.4,1342.4,1344.2,5075
2025-02-07 14:20:00,1344.2,1344.7,1343.6,1344.0,4865
2025-02-07 14:25:00,1344.1,1344.2,1342.8,1343.3,5013


In [5]:
data["ATR"] = ta.atr(data['High'], data['Low'], data['Close'], length=25)
# Calculate the rolling mean of ATR
data['ATR_MA_5'] = data['ATR'].rolling(5).mean()
data['ATR_breakout'] = np.where((data['ATR'] > data['ATR_MA_5']), True, False)
#Calculate the three-candle rolling high
data['max_3_high'] = data['High'].rolling(3).max()
# Check if the fourth candle is higher than the highest of the previous 3 candle
data['is_max_4_high'] = np.where( data['High'] > data['max_3_high'].shift(1), True, False)
# Calculate the three-candle rolling low
data['min_3_low'] = data['Low'].rolling(3).min()
# Check if the fourth candle is lower than the lowest of the previous 3 candles
data['is_min_4_low'] = np.where( data['Low'] < data['min_3_low'].shift(1), True, False)

### Calculate signal

In [6]:
def cal_signal(row):
    signal = ''
    if row['ATR_breakout'] & row['is_max_4_high']:
        signal = 'long'
    elif row['ATR_breakout'] & row['is_min_4_low']:
        signal = 'short'
    return signal
data['signal'] = data.apply(lambda r: cal_signal(r), axis=1)

In [7]:
data

,Open,High,Low,Close,Volume,ATR,ATR_MA_5,ATR_breakout,max_3_high,is_max_4_high,min_3_low,is_min_4_low,signal
Date,,,,,,,,,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812,NaN,NaN,False,NaN,False,NaN,False,
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323,NaN,NaN,False,NaN,False,NaN,False,
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207,NaN,NaN,False,943.6,False,942.6,False,
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196,NaN,NaN,False,943.5,False,942.3,True,
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765,NaN,NaN,False,943.7,True,942.3,False,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-07 14:15:00,1343.3,1344.4,1342.4,1344.2,5075,1.514610,1.465103,True,1346.5,False,1342.4,True,short
2025-02-07 14:20:00,1344.2,1344.7,1343.6,1344.0,4865,1.498025,1.476099,True,1345.2,False,1342.4,False,
2025-02-07 14:25:00,1344.1,1344.2,1342.8,1343.3,5013,1.494104,1.489055,True,1344.7,False,1342.4,False,


In [8]:
from cal_return_v1 import cal_return
data = cal_return(data)
data

,Open,High,Low,Close,Volume,ATR,ATR_MA_5,ATR_breakout,max_3_high,is_max_4_high,min_3_low,is_min_4_low,signal,return,exit_time
Date,,,,,,,,,,,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812,NaN,NaN,False,NaN,False,NaN,False,,,
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323,NaN,NaN,False,NaN,False,NaN,False,,,
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207,NaN,NaN,False,943.6,False,942.6,False,,,
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196,NaN,NaN,False,943.5,False,942.3,True,,,
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765,NaN,NaN,False,943.7,True,942.3,False,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-07 14:15:00,1343.3,1344.4,1342.4,1344.2,5075,1.514610,1.465103,True,1346.5,False,1342.4,True,short,0.9,2025-02-07 14:25:00
2025-02-07 14:20:00,1344.2,1344.7,1343.6,1344.0,4865,1.498025,1.476099,True,1345.2,False,1342.4,False,,,
2025-02-07 14:25:00,1344.1,1344.2,1342.8,1343.3,5013,1.494104,1.489055,True,1344.7,False,1342.4,False,,,


In [9]:
has_return = data[data['return'] != '']

In [10]:
has_return['return'].sum()

6769.799999999976

In [11]:
len(has_return['return'])

16424

In [12]:
has_return[has_return.index > '2024-07-19 08:45:00']

,Open,High,Low,Close,Volume,ATR,ATR_MA_5,ATR_breakout,max_3_high,is_max_4_high,min_3_low,is_min_4_low,signal,return,exit_time
Date,,,,,,,,,,,,,,,
2024-07-19 09:25:00,1303.2,1309.1,1303.2,1309.0,8842,2.218103,2.138883,True,1309.1,True,1302.3,False,long,-3,2024-07-19 10:20:00
2024-07-19 09:30:00,1309.2,1311.2,1308.0,1308.4,8595,2.257379,2.153328,True,1311.2,True,1302.3,False,long,-3,2024-07-19 10:25:00
2024-07-19 13:10:00,1303.0,1303.3,1300.5,1300.7,4046,1.769317,1.761881,True,1303.3,True,1300.5,True,long,-3,2024-07-19 14:10:00
2024-07-19 13:20:00,1300.9,1301.9,1299.7,1301.9,4754,1.760843,1.750310,True,1303.3,False,1299.7,True,short,-0.3,2024-07-19 14:25:00
2024-07-19 13:25:00,1301.7,1303.4,1301.0,1303.4,4121,1.786409,1.757097,True,1303.4,True,1299.7,False,long,-3,2024-07-19 13:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-07 13:30:00,1341.6,1342.7,1341.2,1342.0,3548,1.337839,1.318168,True,1342.7,True,1339.5,False,long,1.3,2025-02-07 14:25:00
2025-02-07 13:35:00,1342.0,1343.9,1342.0,1343.7,5828,1.360326,1.335841,True,1343.9,True,1341.2,False,long,-0.4,2025-02-07 14:25:00
2025-02-07 13:40:00,1343.9,1347.4,1343.8,1345.3,14175,1.453913,1.365608,True,1347.4,True,1341.2,False,long,-2.0,2025-02-07 14:25:00
